<center>CQF Final Project<center>
<center>Atish Patil<center>
<center>January 2023 Cohort<center>
    
### <center>Pairs Trading Strategy Design & Backtest<center>
---

### Contents

- [Libraries and Setup](#libraries-and-setup)
- [Data Preparation](#data-preparation)
- [Analytics Functions](#analytics-functions)
- [Cointegration Analysis](#cointegration-analysis)
- [Strategy Backtest](#strategy-backtest)
- [Trade Metrics](#trade-metrics)
- [Portfolio Optimization](#portfolio-optimization)
---

### Libraries and Setup <a name="libraries-and-setup"></a>

In [ ]:
# !pip install yfinance pandas plotly statsmodels watermark

In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Download data
import yfinance as yf

# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Analytics
from statsmodels.tsa.stattools import adfuller
from itertools import combinations

# Machine info & package version
from watermark import watermark
%load_ext watermark
%watermark -a "Atish Patil" -u -d -v -m -iv

### Data Preparation <a name="data-preparation"></a>

In [ ]:
# List of ETFs to fetch data for
etf_list = ['EWJ', 'MCHI', 'INDA', 'EWZ', 'EWY', 'EWT', 'EWC', 'EWU', 'EWA']

# EWJ - iShares MSCI Japan ETF
# MCHI - iShares MSCI China ETF
# INDA - iShares MSCI India ETF
# EWZ - iShares MSCI Brazil ETF
# EWY - iShares MSCI South Korea ETF
# EWT - iShares MSCI Taiwan ETF
# EWC - iShares MSCI Canada ETF
# EWU - iShares MSCI United Kingdom ETF
# EWA - iShares MSCI Australia ETF

# Function to get ETF data from Yahoo Finance
def fetch_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

# Dictionary to hold ETF data
etf_data = {}

# Get data for each ETF and store in the dictionary
start_date = '2019-01-01'
end_date = '2023-07-31'
for etf_symbol in etf_list:
    data = fetch_data(etf_symbol, start_date, end_date)
    etf_data[etf_symbol] = data

In [ ]:
# Display the fetched data for each ETF
for etf_symbol, data in etf_data.items():
    print(f'ETF: {etf_symbol}')
    print(data)

In [ ]:
# Create an interactive Plotly figure for close prices
fig_close_prices = go.Figure()

# Add lines for each ETF's close prices
for etf_symbol, data in etf_data.items():
    fig_close_prices.add_trace(go.Scatter(x=data.index, y=data['Adj Close'], mode='lines', name=etf_symbol))

# Customize the layout
fig_close_prices.update_layout(title='Close Prices of ETFs',
                               xaxis=dict(rangeslider=dict(visible=False), type='date'),
                               yaxis_title='Close Price',
                               xaxis_title='Date',
                               legend_title='ETF',
                               height=700,
                               width=900,
                               showlegend=True)

# Show the Plotly figure for close prices
fig_close_prices.show()

In [ ]:
# Create an interactive Plotly figure for daily returns
fig_daily_returns = go.Figure()

# Calculate and plot daily returns for each ETF
for etf_symbol, data in etf_data.items():
    daily_returns = data['Adj Close'].pct_change().dropna()
    fig_daily_returns.add_trace(go.Scatter(x=daily_returns.index, y=daily_returns, mode='lines', name=etf_symbol))

# Customize the layout
fig_daily_returns.update_layout(title='Daily Returns of ETFs',
                                xaxis=dict(rangeslider=dict(visible=False), type='date'),
                                yaxis_title='Daily Return',
                                xaxis_title='Date',
                                legend_title='ETF',
                                height=700,
                                width=900,
                                showlegend=True)

# Show the Plotly figure for daily returns
fig_daily_returns.show()

### Analytics Functions <a name="analytics-functions"></a>

In [ ]:
def ols_regression(y, X):
    """
    Perform OLS regression in matrix form.

    y (numpy array or pandas Series): The dependent variable.
    X (numpy array or pandas DataFrame): The independent variables (design matrix).
    """
    # If inputs are pandas Series, convert them to numpy arrays
    if isinstance(y, pd.Series):
        y = y.values
    if isinstance(X, pd.DataFrame):
        X = X.values

    # Add a constant column to the design matrix (for the intercept term)
    X = np.column_stack((np.ones(len(X)), X))

    # Calculate OLS estimated coefficients (beta_hat = (X^T * X)^-1 * X^T * y)
    beta_hat = np.linalg.inv(X.T @ X) @ X.T @ y

    return beta_hat

In [ ]:
def adf_test(series):
    """
    Perform Augmented Dickey-Fuller (ADF) test for stationarity.

    series (numpy array or pandas Series): The time series data.
    """
    result = adfuller(series, autolag='AIC')
    return result[1]

In [ ]:
def ols_and_adf_test(y, X):
    """
    Perform OLS regression and ADF test for stationarity.

    y (pandas Series): The dependent variable.
    X (pandas Series): The independent variable.
    """
    # Perform OLS regression to obtain residuals
    beta_hat = ols_regression(y, X)
    residuals = y - (beta_hat[0] + beta_hat[1] * X)

    # Test for stationarity of residuals using ADF test
    p_value = adf_test(residuals)

    return beta_hat[1], p_value

In [ ]:
def engle_granger_cointegration_test(y, X):
    """
    Perform the Engle-Granger two-step cointegration test.

    y (pandas Series): The dependent variable.
    X (pandas Series): The independent variable.
    """
    # Perform OLS regression to obtain residuals
    beta_hat = ols_regression(y, X)
    residuals = y - (beta_hat[0] + beta_hat[1] * X)

    # Test for stationarity of residuals using ADF test
    p_value = adf_test(residuals)

    # Return the p-value of the Engle-Granger test
    return p_value

In [ ]:
def calculate_spread(pair1, pair2):
    # Calculate the spread using OLS regression for pair1 and pair2
    spread_data = pd.DataFrame()
    spread_data[pair1] = etf_data[pair1]['Adj Close']
    spread_data[pair2] = etf_data[pair2]['Adj Close']

    # Perform OLS regression to find residuals (spread)
    beta_hat = ols_regression(spread_data[pair1], spread_data[pair2])
    spread_data['spread'] = spread_data[pair1] - beta_hat[0] - beta_hat[1] * spread_data[pair2]

    # Create an interactive Plotly figure
    fig = go.Figure()

    # Plot the spread using Plotly
    fig.add_trace(go.Scatter(x=spread_data.index, y=spread_data['spread'], mode='lines', 
                             name='Spread', line=dict(color='green')))

    # Customize the layout
    fig.update_layout(title=f'Spread between {pair1} and {pair2}',
                      xaxis_title='Date',
                      yaxis_title='Spread',
                      xaxis=dict(rangeslider=dict(visible=False), type='date'),
                      showlegend=True,
                      template='plotly',
                      xaxis_showgrid=False,
                      yaxis_showgrid=True,
                      yaxis_zeroline=False,
                      height=600,
                      width=900)

    # Show the Plotly figure
    fig.show()
    
    return spread_data

In [ ]:
# Define the mean reversion strategy
def mean_reversion_strategy(df, lookback, std_dev):
    """
    Implement a mean reversion trading strategy using Bollinger Bands.

    df (pandas DataFrame): The DataFrame containing 'spread' column for the spread data.
    lookback (int): The window size for calculating moving average and standard deviation.
    std_dev (float): The number of standard deviations for the Bollinger Bands.
    """
    
    # Moving Average
    df['moving_average'] = df.spread.rolling(lookback).mean()
    print(df)  # Print DataFrame after moving average calculation
    
    # Moving Standard Deviation
    df['moving_std_dev'] = df.spread.rolling(lookback).std()

    # Upper band and lower band
    df['upper_band'] = df.moving_average + std_dev * df.moving_std_dev
    df['lower_band'] = df.moving_average - std_dev * df.moving_std_dev

    # Long positions
    df['long_entry'] = df.spread < df.lower_band
    df['long_exit'] = df.spread >= df.moving_average

    df['positions_long'] = np.nan
    df.loc[df.long_entry, 'positions_long'] = 1
    df.loc[df.long_exit, 'positions_long'] = 0
    df.positions_long = df.positions_long.fillna(method='ffill')

    # Short positions
    df['short_entry'] = df.spread > df.upper_band
    df['short_exit'] = df.spread <= df.moving_average

    df['positions_short'] = np.nan
    df.loc[df.short_entry, 'positions_short'] = -1
    df.loc[df.short_exit, 'positions_short'] = 0

    df.positions_short = df.positions_short.fillna(method='ffill')

    # Positions
    df['positions'] = df.positions_long + df.positions_short

    return df

In [ ]:
def calc_drawdown(cum_rets):
    """
    Calculate the drawdown (percentage drop from peak) of cumulative returns.
    
    cum_rets (pandas Series): Cumulative returns series.
    """

    # Calculate the running maximum
    running_max = np.maximum.accumulate(cum_rets.dropna())
    # Ensure the value never drops below 1
    running_max[running_max < 1] = 1
    # Calculate the percentage drawdown
    drawdown = (cum_rets)/running_max - 1
    
    return drawdown

def calculate_metrics(df, pair1, pair2):
    """
    Calculate strategy performance metrics.
    
    df (pandas DataFrame): DataFrame containing strategy-related columns.
    """

    df['percentage_change'] = (df.spread - df.spread.shift(1)) / (beta_hat[1] * df[pair1] + df[pair2])
    df['strategy_returns'] = df.positions.shift(1) * df.percentage_change
    df["cumulative_returns"] = (df.strategy_returns + 1).cumprod()
    cumulative_returns = (df['cumulative_returns'].iloc[-1] - 1) * 100
    
    sharpe_ratio = np.mean(df['strategy_returns']) / np.std(df['strategy_returns']) * (252 ** 0.5)
    
    max_drawdown = calc_drawdown(df["cumulative_returns"]).min() * 100

    return cumulative_returns, sharpe_ratio, max_drawdown

def plot_drawdown(drawdown):
    """
    Plot the drawdown of cumulative returns.

    drawdown (pandas Series): Drawdown values to plot.
    """

    fig = go.Figure()

    # Plot
    fig.add_trace(go.Scatter(x=drawdown.index, y=drawdown, fill='tozeroy', fillcolor='rgba(255, 0, 0, 0.3)',
                             line=dict(color='red'),
                             name='Drawdown'))

    fig.update_layout(title='Drawdown',
                      xaxis_title='Date',
                      yaxis_title='Returns',
                      xaxis=dict(rangeslider=dict(visible=False), type='date'),
                      showlegend=False,
                      template='plotly',
                      xaxis_showgrid=False,
                      yaxis_showgrid=True,
                      yaxis_zeroline=False,
                      height=700,
                      width=900)

    fig.show()

In [ ]:
def parameter_optimization(spread_data, pair1, pair2):
    # Perform parameter optimization
    best_returns = -np.inf
    best_lookback = None
    best_std_dev = None

    # Loop through different parameters for optimization
    for lookback in range(10, 100, 5):
        for std_dev in np.arange(1.0, 5.0, 0.5):
            # Calculate strategy and metrics with current parameters
            df = mean_reversion_strategy(spread_data, lookback, std_dev)
            cumulative_returns, sharpe_ratio, max_drawdown = calculate_metrics(df, pair1, pair2)

            # Update best parameters if better metrics found
            if cumulative_returns > best_returns:
                best_returns = cumulative_returns
                best_lookback = lookback
                best_std_dev = std_dev
    
    return best_lookback, best_std_dev

### Cointegration Analysis <a name="cointegration-analysis"></a>

In [ ]:
# Create a list of all possible pairs of ETFs
all_pairs = list(combinations(etf_data.keys(), 2))

# Dictionary to hold cointegration test results
cointegration_p_values = {}

# Perform OLS Regression and Engle-Granger Cointegration Test for Each Pair
for pair in all_pairs:
    index_1, index_2 = pair

    # Get the closing prices for both ETFs
    y = etf_data[index_1]['Adj Close']
    X = etf_data[index_2]['Adj Close']

    # Calculate OLS regression coefficients
    beta_hat = ols_regression(y, X)

    # Calculate the difference between actual and estimated values (residuals)
    residuals = y - (beta_hat[0] + beta_hat[1] * X)

    # Test for cointegration using Engle-Granger test
    p_value = engle_granger_cointegration_test(y, X)

    # Store cointegration test result for both pair directions
    cointegration_p_values[(index_1, index_2)] = p_value
    cointegration_p_values[(index_2, index_1)] = p_value

    # Display information about the pair and the cointegration test
    print(f"Pair: {index_1} - {index_2}")
    print(f"Beta_hat: {beta_hat}")
    print(f"Cointegration test p-value: {round(p_value, 6)}")

    # Determine if the pair is likely cointegrated based on the p-value
    if p_value <= 0.05:
        print("The pair is likely cointegrated.")
    else:
        print("The pair is not cointegrated.")

    print("------------------------------------------")

In [ ]:
# Filter cointegrated pairs and sort by p-value
cointegrated_pairs = {pair: p_value for pair, p_value in cointegration_p_values.items() if p_value <= 0.05}
unique_cointegrated_pairs = {}

for (index_1, index_2), p_value in cointegrated_pairs.items():
    sorted_pair = tuple(sorted([index_1, index_2]))
    if sorted_pair not in unique_cointegrated_pairs or p_value < unique_cointegrated_pairs[sorted_pair]:
        unique_cointegrated_pairs[sorted_pair] = p_value

sorted_unique_cointegrated_pairs = sorted(unique_cointegrated_pairs.items(), key=lambda item: item[1])

# Print cointegrated pairs sorted by p-value, unique, and count
print("Cointegrated Pairs (Sorted by p-value, Unique):")
print("-----------------------------------------------")
for (index_1, index_2), p_value in sorted_unique_cointegrated_pairs:
    print(f"Pair: {index_1} - {index_2}")
    print(f"Cointegration test p-value: {round(p_value, 6)}")
    print("-----------------------------------------------")

In [ ]:
cointegrated_count = len(sorted_unique_cointegrated_pairs)
print(f"Cointegrated pairs count: {cointegrated_count}")

In [ ]:
# Create a matrix to store p-values
p_values_matrix = np.zeros((len(etf_data), len(etf_data)))

# Calculate and fill the matrix with cointegration test p-values
for i, etf1 in enumerate(etf_data):
    for j, etf2 in enumerate(etf_data):
        if i != j:
            p_value = cointegration_p_values[(etf1, etf2)]
            p_values_matrix[i, j] = p_value

# Set up the heatmap using Plotly
fig = go.Figure(data=go.Heatmap(z=p_values_matrix,
                                 x=list(etf_data.keys()),
                                 y=list(etf_data.keys()),
                                 colorscale='Viridis'))

# Add p-values as annotations on the heatmap
annotations = []
for i, etf1 in enumerate(etf_data):
    for j, etf2 in enumerate(etf_data):
        if i != j:
            annotations.append(dict(x=etf2, y=etf1, text=f'{p_values_matrix[i, j]:.3f}',
                                    font=dict(color='white'), showarrow=False))

fig.update_layout(title='Cointegration Test Results - Heatmap',
                  xaxis_title='ETFs',
                  yaxis_title='ETFs',
                  xaxis=dict(tickangle=45),
                  yaxis=dict(tickangle=0),
                  height=700,
                  width=900,
                  annotations=annotations)

# Show the Plotly figure
fig.show()

### Strategy Backtest <a name="strategy-backtest"></a>

In [ ]:
ewc_inda_spread_data = calculate_spread("EWC", "INDA")

In [ ]:
# Call the mean_reversion_strategy function
ewc_inda_df = mean_reversion_strategy(ewc_inda_spread_data, 30, 2)

In [ ]:
# Calculate strategy performance metrics
ewc_inda_cumulative_returns, ewc_inda_sharpe_ratio, ewc_inda_max_drawdown = calculate_metrics(ewc_inda_df, "EWC", "INDA")

# Print the calculated metrics
print("EWC_INDA strategy returns before parameter optimization")
print("-------------------------------------------------------")
print("Strategy Returns(%):", round(ewc_inda_cumulative_returns, 2))
print("Sharpe Ratio:", round(ewc_inda_sharpe_ratio, 2))
print("Max Drawdown(%):", round(ewc_inda_max_drawdown, 2))

In [ ]:
# Create a Plotly figure for cumulative returns
ewc_inda_cum_ret = go.Figure()

# Plot cumulative returns using Plotly
ewc_inda_cum_ret.add_trace(go.Scatter(x=ewc_inda_df.index, y=ewc_inda_df.cumulative_returns, mode='lines', 
                                 name='Returns', line=dict(color='magenta')))

# Customize the layout
ewc_inda_cum_ret.update_layout(title='Cumulative Returns',
                              xaxis_title='Date',
                              yaxis_title='Cumulative Returns',
                              height=700,
                              width=900)

# Show the Plotly figure
ewc_inda_cum_ret.show()

In [ ]:
# Calculate drawdown
ewc_inda_drawdown = calc_drawdown(ewc_inda_df.cumulative_returns)

# Print maximum drawdown
print("The maximum drawdown is %.2f" % (ewc_inda_drawdown.min() * 100))

# Plot drawdown
plot_drawdown(ewc_inda_drawdown)

In [ ]:
# Call the function for parameter optimization
ewc_inda_best_lookback, ewc_inda_best_std_dev = parameter_optimization(ewc_inda_spread_data, "EWC", "INDA")

In [ ]:
# Print the best parameters found during optimization
print(f"Lookback: {ewc_inda_best_lookback}, Std Dev: {ewc_inda_best_std_dev}")

In [ ]:
# Call the mean_reversion_strategy function
ewc_inda_df2 = mean_reversion_strategy(ewc_inda_spread_data, 10, 1)

In [ ]:
# Calculate strategy performance metrics
ewc_inda_cumulative_returns_2, ewc_inda_sharpe_ratio_2, ewc_inda_max_drawdown_2 = calculate_metrics(ewc_inda_df2, "EWC", "INDA")

# Print the calculated metrics
print("EWC_INDA strategy returns after parameter optimization")
print("-------------------------------------------------------")
print("Strategy Returns(%):", round(ewc_inda_cumulative_returns_2, 2))
print("Sharpe Ratio:", round(ewc_inda_sharpe_ratio_2, 2))
print("Max Drawdown(%):", round(ewc_inda_max_drawdown_2, 2))

In [ ]:
# Create a Plotly figure for cumulative returns
ewc_inda_cum_ret_2 = go.Figure()

# Plot cumulative returns using Plotly
ewc_inda_cum_ret_2.add_trace(go.Scatter(x=ewc_inda_df2.index, y=ewc_inda_df2.cumulative_returns, mode='lines', 
                                 name='Returns', line=dict(color='magenta')))

# Customize the layout
ewc_inda_cum_ret_2.update_layout(title='Cumulative Returns',
                                xaxis_title='Date',
                                yaxis_title='Cumulative Returns',
                                height=700,
                                width=900)

# Show the Plotly figure
ewc_inda_cum_ret_2.show()

In [ ]:
# Calculate drawdown
ewc_inda_drawdown_2 = calc_drawdown(ewc_inda_df2.cumulative_returns)

# Print maximum drawdown
print("The maximum drawdown is %.2f" % (ewc_inda_drawdown_2.min() * 100))

# Plot drawdown
plot_drawdown(ewc_inda_drawdown_2)

In [ ]:
ewj_ewy_spread_data = calculate_spread("EWJ", "EWY")

In [ ]:
# Call the mean_reversion_strategy function
ewj_ewy_df = mean_reversion_strategy(ewj_ewy_spread_data, 30, 2)

In [ ]:
# Calculate strategy performance metrics
ewj_ewy_cumulative_returns, ewj_ewy_sharpe_ratio, ewj_ewy_max_drawdown = calculate_metrics(ewj_ewy_df, "EWJ", "EWY")

# Print the calculated metrics
print("EWJ_EWY strategy returns before parameter optimization")
print("-------------------------------------------------------")
print("Strategy Returns(%):", round(ewj_ewy_cumulative_returns, 2))
print("Sharpe Ratio:", round(ewj_ewy_sharpe_ratio, 2))
print("Max Drawdown(%):", round(ewj_ewy_max_drawdown, 2))

In [ ]:
# Create a Plotly figure for cumulative returns
ewj_ewy_cum_ret = go.Figure()

# Plot cumulative returns using Plotly
ewj_ewy_cum_ret.add_trace(go.Scatter(x=ewj_ewy_df.index, y=ewj_ewy_df.cumulative_returns, mode='lines', 
                                 name='Returns', line=dict(color='magenta')))

# Customize the layout
ewj_ewy_cum_ret.update_layout(title='Cumulative Returns',
                             xaxis_title='Date',
                             yaxis_title='Cumulative Returns',
                             height=700,
                             width=900)

# Show the Plotly figure
ewj_ewy_cum_ret.show()

In [ ]:
# Calculate drawdown
ewj_ewy_drawdown = calc_drawdown(ewj_ewy_df.cumulative_returns)

# Print maximum drawdown
print("The maximum drawdown is %.2f" % (ewj_ewy_drawdown.min() * 100))

# Plot drawdown
plot_drawdown(ewj_ewy_drawdown)

In [ ]:
# Call the function for parameter optimization
ewj_ewy_best_lookback, ewj_ewy_best_std_dev = parameter_optimization(ewj_ewy_spread_data, "EWJ", "EWY")

In [ ]:
# Print the best parameters found during optimization
print(f"Lookback: {ewj_ewy_best_lookback}, Std Dev: {ewj_ewy_best_std_dev}")

In [ ]:
# Call the mean_reversion_strategy function
ewj_ewy_df2 = mean_reversion_strategy(ewj_ewy_spread_data, 25, 1.5)

In [ ]:
# Calculate strategy performance metrics
ewj_ewy_cumulative_returns_2, ewj_ewy_sharpe_ratio_2, ewj_ewy_max_drawdown_2 = calculate_metrics(ewj_ewy_df2, "EWJ", "EWY")

# Print the calculated metrics
print("EWJ_EWY strategy returns after parameter optimization")
print("-------------------------------------------------------")
print("Strategy Returns(%):", round(ewj_ewy_cumulative_returns_2, 2))
print("Sharpe Ratio:", round(ewj_ewy_sharpe_ratio_2, 2))
print("Max Drawdown(%):", round(ewj_ewy_max_drawdown_2, 2))

In [ ]:
# Create a Plotly figure for cumulative returns
ewj_ewy_cum_ret_2 = go.Figure()

# Plot cumulative returns using Plotly
ewj_ewy_cum_ret_2.add_trace(go.Scatter(x=ewj_ewy_df2.index, y=ewj_ewy_df2.cumulative_returns, mode='lines', 
                                 name='Returns', line=dict(color='magenta')))

# Customize the layout
ewj_ewy_cum_ret_2.update_layout(title='Cumulative Returns',
                              xaxis_title='Date',
                              yaxis_title='Cumulative Returns',
                              height=700,
                              width=900)

# Show the Plotly figure
ewj_ewy_cum_ret_2.show()

In [ ]:
# Calculate drawdown
ewj_ewy_drawdown_2 = calc_drawdown(ewj_ewy_df2.cumulative_returns)

# Print maximum drawdown
print("The maximum drawdown is %.2f" % (ewj_ewy_drawdown_2.min() * 100))

# Plot drawdown
plot_drawdown(ewj_ewy_drawdown_2)

In [ ]:
# Create a new dataframe to store daily returns
daily_returns_df = pd.DataFrame(index=ewc_inda_df2.index)

# Extract and store daily returns from Strategy 1 (ewc_inda)
daily_returns_df['ewc_inda'] = ewc_inda_df2['strategy_returns']

# Extract and store daily returns from Strategy 2 (inda_ivv)
daily_returns_df['ewj_ewy'] = ewj_ewy_df2['strategy_returns']

# Print the new dataframe
print(daily_returns_df)

In [ ]:
# Calculate cumulative returns for each strategy
daily_returns_df['cumulative_ewc_inda'] = (daily_returns_df['ewc_inda'] + 1).cumprod()
daily_returns_df['cumulative_ewj_ewy'] = (daily_returns_df['ewj_ewy'] + 1).cumprod()

# Create a Plotly figure for cumulative returns
daily_cum_ret_fig = go.Figure()

# Plot cumulative returns using Plotly
daily_cum_ret_fig.add_trace(go.Scatter(x=daily_returns_df.index, y=daily_returns_df['cumulative_ewc_inda'], 
                                       mode='lines', name='EWC-INDA Strategy'))
daily_cum_ret_fig.add_trace(go.Scatter(x=daily_returns_df.index, y=daily_returns_df['cumulative_ewj_ewy'], 
                                       mode='lines', name='EWJ-EWY Strategy'))

# Customize the layout
daily_cum_ret_fig.update_layout(title='Cumulative Returns of Mean Reversion Strategies',
                                xaxis_title='Date',
                                yaxis_title='Cumulative Returns',
                                height=700,
                                width=900)

# Show the Plotly figure
daily_cum_ret_fig.show()

In [ ]:
# Resample daily returns to monthly returns
monthly_returns_df = daily_returns_df.resample('M').sum() * 100

# Pivot the monthly returns dataframe to have years on y-axis and months on x-axis
pivot_df = monthly_returns_df.pivot_table(index=monthly_returns_df.index.year,
                                          columns=monthly_returns_df.index.month,
                                          values=['ewc_inda', 'ewj_ewy'])

# Create a subplot with two heatmaps
heatmap_fig = make_subplots(rows=2, cols=1, subplot_titles=['EWC-INDA Strategy', 'EWJ-EWY Strategy'])

# Create heatmap for EWC-INDA Strategy
heatmap_ewc_inda = go.Heatmap(z=pivot_df['ewc_inda'].values,
                              x=list(range(1, 13)),
                              y=pivot_df.index,
                              colorscale='RdYlGn',
                              colorbar=dict(title='Returns'),
                              hoverinfo='z',
                              showscale=False)

# Create heatmap for EWJ-EWY Strategy
heatmap_ewj_ewy = go.Heatmap(z=pivot_df['ewj_ewy'].values,
                             x=list(range(1, 13)),
                             y=pivot_df.index,
                             colorscale='RdYlGn',
                             colorbar=dict(title='Returns'),
                             hoverinfo='z',
                             showscale=False)

# Add heatmaps to the subplots
heatmap_fig.add_trace(heatmap_ewc_inda, row=1, col=1)
heatmap_fig.add_trace(heatmap_ewj_ewy, row=2, col=1)

# Add annotations to the correct subplots
heatmap_fig.add_trace(go.Scatter(x=[], y=[]))  # Dummy trace to reserve space for annotations
annotations_ewc = []
for i, row in enumerate(pivot_df['ewc_inda'].values):
    for j, value in enumerate(row):
        annotations_ewc.append(
            dict(
                x=j + 1,
                y=pivot_df.index[i],
                text='{:.2f}'.format(value),
                showarrow=False,
                font=dict(color='black')
            )
        )

annotations_ewj = []
for i, row in enumerate(pivot_df['ewj_ewy'].values):
    for j, value in enumerate(row):
        annotations_ewj.append(
            dict(
                x=j + 1,
                y=pivot_df.index[i],
                text='{:.2f}'.format(value),
                showarrow=False,
                font=dict(color='black')
            )
        )

for annotation in annotations_ewc:
    heatmap_fig.add_annotation(annotation, row=1, col=1)

for annotation in annotations_ewj:
    heatmap_fig.add_annotation(annotation, row=2, col=1)

# Update layout and axis labels
heatmap_fig.update_layout(
    width=900,   # Set the width of the heatmap_figure in pixels
    height=700,  # Set the height of the heatmap_figure in pixels
    title='Monthly Returns Heatmap of Mean Reversion Strategies')
month_names = pd.date_range(start='2023-01-01', periods=12, freq='M').strftime('%B')
heatmap_fig.update_xaxes(tickvals=list(range(1, 13)), ticktext=month_names)

# Display the subplots
heatmap_fig.show()

### Trade Metrics  <a name="trade-metrics"></a>

In [ ]:
ewc_inda_returns = daily_returns_df["ewc_inda"]
ewj_ewy_returns = daily_returns_df["ewj_ewy"]

# Calculate total wins, total losses, total trades for ewc_inda
ewc_inda_total_wins = round(sum(1 for ret in ewc_inda_returns if ret > 0))
ewc_inda_total_losses = round(sum(1 for ret in ewc_inda_returns if ret < 0))
ewc_inda_total_trades = ewc_inda_total_wins + ewc_inda_total_losses

# Calculate total wins, total losses, total trades for ewj_ewy
ewj_ewy_total_wins = round(sum(1 for ret in ewj_ewy_returns if ret > 0))
ewj_ewy_total_losses = round(sum(1 for ret in ewj_ewy_returns if ret < 0))
ewj_ewy_total_trades = ewj_ewy_total_wins + ewj_ewy_total_losses

# Calculate win ratio and loss ratio for ewc_inda
ewc_inda_win_ratio = ewc_inda_total_wins / ewc_inda_total_trades
ewc_inda_loss_ratio = ewc_inda_total_losses / ewc_inda_total_trades

# Calculate win ratio and loss ratio for ewj_ewy
ewj_ewy_win_ratio = ewj_ewy_total_wins / ewj_ewy_total_trades
ewj_ewy_loss_ratio = ewj_ewy_total_losses / ewj_ewy_total_trades

# Calculate max profit and max loss for ewc_inda and ewj_ewy
ewc_inda_max_profit = np.nanmax(ewc_inda_returns) * 100
ewc_inda_max_loss = np.nanmin(ewc_inda_returns) * 100
ewj_ewy_max_profit = np.nanmax(ewj_ewy_returns) * 100
ewj_ewy_max_loss = np.nanmin(ewj_ewy_returns) * 100

# Create a dictionary to store the calculated metrics
data = {
    'Metrics': [
        'total_wins', 'total_losses', 'total_trades', 
        'win_ratio', 'loss_ratio', 'max_profit', 'max_loss'
    ],
    'EWC_INDA': [
        int(ewc_inda_total_wins), int(ewc_inda_total_losses), int(ewc_inda_total_trades), 
        round(ewc_inda_win_ratio, 3), round(ewc_inda_loss_ratio, 3), 
        round(ewc_inda_max_profit, 3), round(ewc_inda_max_loss, 3)
    ],
    'EWJ_EWY': [
        int(ewj_ewy_total_wins), int(ewj_ewy_total_losses), int(ewj_ewy_total_trades), 
        round(ewj_ewy_win_ratio, 3), round(ewj_ewy_loss_ratio, 3), 
        round(ewj_ewy_max_profit, 3), round(ewj_ewy_max_loss, 3)
    ]
}

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame(data)
metrics_df.set_index('Metrics', inplace=True)

# Display the calculated metrics DataFrame
metrics_df

In [ ]:
# List of ETFs to fetch data for
etf_pairs_list = ['EWC', 'INDA', 'EWJ', 'EWY']

# Dictionary to hold ETF data
etf_pairs_data = {}

for etf_symbol in etf_pairs_list:
    data = fetch_data(etf_symbol, start_date, end_date)
    etf_pairs_data[etf_symbol] = data

In [ ]:
for etf_symbol, data in etf_pairs_data.items():
    data["Returns"] = data['Adj Close'].pct_change().dropna()
    data['cum_ret'] = (data['Returns'] + 1).cumprod()
    print(data)

In [ ]:
# Returns
ewc_inda_ret = round((((daily_returns_df['ewc_inda'] + 1).cumprod().iloc[-1]) - 1) * 100, 2)
ewc_ret = round((etf_pairs_data['EWC']['cum_ret'].iloc[-1] - 1) * 100, 2)
inda_ret = round((etf_pairs_data['INDA']['cum_ret'].iloc[-1] - 1) * 100, 2)

# Risk
ewc_inda_risk = round((np.std(daily_returns_df['ewc_inda']) * np.sqrt(252)) * 100, 2)
ewc_risk = round((np.std(etf_pairs_data['EWC']['Returns'])) * np.sqrt(252) * 100, 2)
inda_risk = round((np.std(etf_pairs_data['INDA']['Returns'])) * np.sqrt(252) * 100, 2)

# Create a dictionary to store the metrics
ret_risk_data = {
    'Metrics': [
        'returns', 'risk'
    ],
    'EWC_INDA': [
        ewc_inda_ret, ewc_inda_risk
    ],
    'EWC': [
        ewc_ret, ewc_risk
    ],
    'INDA': [
        inda_ret, inda_risk
    ]
}

# Create a DataFrame to store the metrics
ret_risk_df = pd.DataFrame(ret_risk_data)
ret_risk_df.set_index('Metrics', inplace=True)

# Display the calculated metrics DataFrame
ret_risk_df

In [ ]:
# Create traces for each ETF's cumulative returns
trace_ewc = go.Scatter(x=data.index, y=etf_pairs_data['EWC']['cum_ret'], mode='lines', name='EWC')
trace_inda = go.Scatter(x=data.index, y=etf_pairs_data['INDA']['cum_ret'], mode='lines', name='INDA')

# Create a trace for the combined cumulative returns
trace_ewc_inda_combined = go.Scatter(x=daily_returns_df.index, y=daily_returns_df['cumulative_ewc_inda'], 
                            mode='lines', name='EWC_INDA_STRAT')

# Create a layout for the plot
layout1 = go.Layout(title='Cumulative Returns of EWC, INDA, and EWC_INDA Strategy',
                   xaxis=dict(title='Date'),
                   yaxis=dict(title='Cumulative Returns'),
                   height=700,
                   width=900)

# Create a figure and add the traces and layout
ewc_inda_bnh_fig = go.Figure(data=[trace_ewc, trace_inda, trace_ewc_inda_combined], layout=layout1)

# Display the plot
ewc_inda_bnh_fig.show()

In [ ]:
# Returns
ewj_ewy_ret = round((((daily_returns_df['ewj_ewy'] + 1).cumprod().iloc[-1]) - 1) * 100, 2)
ewj_ret = round((etf_pairs_data['EWJ']['cum_ret'].iloc[-1] - 1) * 100, 2)
ewy_ret = round((etf_pairs_data['EWY']['cum_ret'].iloc[-1] - 1) * 100, 2)

# Risk
ewj_ewy_risk = round((np.std(daily_returns_df['ewj_ewy']) * np.sqrt(252)) * 100, 2)
ewj_risk = round((np.std(etf_pairs_data['EWJ']['Returns'])) * np.sqrt(252) * 100, 2)
ewy_risk = round((np.std(etf_pairs_data['EWY']['Returns'])) * np.sqrt(252) * 100, 2)

# Create a dictionary to store the metrics
ret_risk_data_2 = {
    'Metrics': [
        'returns', 'risk'
    ],
    'EWJ_EWY': [
        ewj_ewy_ret, ewj_ewy_risk
    ],
    'EWJ': [
        ewj_ret, ewj_risk
    ],
    'EWY': [
        ewy_ret, ewy_risk
    ]
}

# Create a DataFrame to store the metrics
ret_risk_df_2 = pd.DataFrame(ret_risk_data_2)
ret_risk_df_2.set_index('Metrics', inplace=True)

# Display the calculated metrics DataFrame
ret_risk_df_2

In [ ]:
# Create traces for each ETF's cumulative returns
trace_ewj = go.Scatter(x=data.index, y=etf_pairs_data['EWJ']['cum_ret'], mode='lines', name='EWJ')
trace_ewy = go.Scatter(x=data.index, y=etf_pairs_data['EWY']['cum_ret'], mode='lines', name='EWY')

# Create a trace for the combined cumulative returns
trace_ewj_ewy_combined = go.Scatter(x=daily_returns_df.index, y=daily_returns_df['cumulative_ewj_ewy'], 
                            mode='lines', name='EWJ_EWY_STRAT')

# Create a layout for the plot
layout2 = go.Layout(title='Cumulative Returns of EWJ, EWY, and EWJ_EWY Strategy',
                   xaxis=dict(title='Date'),
                   yaxis=dict(title='Cumulative Returns'),
                   height=700,
                   width=900)

# Create a figure and add the traces and layout
ewj_ewy_bnh_fig = go.Figure(data=[trace_ewj, trace_ewy, trace_ewj_ewy_combined], layout=layout2)

# Display the plot
ewj_ewy_bnh_fig.show()

In [ ]:
snp500 = fetch_data("^GSPC", start_date, end_date)
snp500["Returns"] = snp500['Adj Close'].pct_change().dropna()
snp500['cum_ret'] = (snp500['Returns'] + 1).cumprod()
snp500

In [ ]:
# Returns
snp500_ret = round(((snp500['Returns'] + 1).cumprod().iloc[-1] - 1) * 100, 2)

# Risk
snp500_risk = round((np.std(snp500['Returns']) * np.sqrt(252)) * 100, 2)

# Create a dictionary to store the metrics
ret_risk_data_3 = {
    'Metrics': [
        'returns', 'risk'
    ],
    'S&P500': [
        snp500_ret, snp500_risk
    ],
    'EWC_INDA': [
        ewc_inda_ret, ewc_inda_risk
    ],
    'EWJ_EWY': [
        ewj_ewy_ret, ewj_ewy_risk
    ]
}

# Create a DataFrame to store the metrics
ret_risk_df_3 = pd.DataFrame(ret_risk_data_3)
ret_risk_df_3.set_index('Metrics', inplace=True)

# Display the calculated metrics DataFrame
ret_risk_df_3

In [ ]:
# Create traces for each strategies
trace_ewc_inda = go.Scatter(x=daily_returns_df.index, y=daily_returns_df['cumulative_ewc_inda'], 
                            mode='lines', name='EWC_INDA_STRAT')
trace_ewj_ewy = go.Scatter(x=daily_returns_df.index, y=daily_returns_df['cumulative_ewj_ewy'], 
                            mode='lines', name='EWJ_EWY_STRAT')

# Create a trace for the S&P500 cumulative returns
trace_snp500 = go.Scatter(x=snp500.index, y=snp500['cum_ret'], mode='lines', name='S&P500')

# Create a layout for the plot
layout3 = go.Layout(title='Cumulative Returns of S&P500, EWC_INDA Strategy and EWJ_EWY Strategy',
                   xaxis=dict(title='Date'),
                   yaxis=dict(title='Cumulative Returns'),
                   height=700,
                   width=900)

# Create a figure and add the traces and layout
snp500_combined = go.Figure(data=[trace_ewc_inda, trace_ewj_ewy, trace_snp500], layout=layout3)

# Display the plot
snp500_combined.show()

### Portfolio Optimization <a name="portfolio-optimization"></a>

In [ ]:
portfolio_df = daily_returns_df.copy()
portfolio_df = portfolio_df[["ewc_inda", "ewj_ewy"]]
portfolio_df["ewc_inda"] = portfolio_df["ewc_inda"]
portfolio_df["ewj_ewy"] = portfolio_df["ewj_ewy"]
portfolio_df

In [ ]:
returns = portfolio_df

In [ ]:
# Define the number of portfolios and assets
numofportfolio = 5000
numofasset = len(returns.columns)

In [ ]:
# Calculate annualized return and volatility
def calculate_annualized_metrics(returns):
    ann_ret = returns.mean() * 252 * 100
    ann_vol = returns.std() * np.sqrt(252) * 100
    return ann_ret, ann_vol

ann_ret, ann_vol = calculate_annualized_metrics(returns)

In [ ]:
# Create a DataFrame for annualized return and volatility
bar_pie_df = pd.DataFrame({
    'Ann Ret': round(ann_ret, 2),
    'Ann Vol': round(ann_vol, 2)
})

# Plot bar chart for annualized return and volatility using Plotly Express
fig_bar = px.bar(
    bar_pie_df,
    x=bar_pie_df.index,
    y=['Ann Ret', 'Ann Vol'],
    title='Annualized Return & Volatility (%)'
)
fig_bar.show()

In [ ]:
# Reset index for the DataFrame
bar_pie_df_reset = bar_pie_df.reset_index()

# Plot pie chart for annualized return using Plotly Express
fig_pie = px.pie(
    bar_pie_df_reset,
    names='index',
    values='Ann Ret',
    title='Annualized Return (%)',
    hole=0.4
)
fig_pie.show()

In [ ]:
def portfolio_simulation(returns):
    """
    Simulate portfolios and calculate portfolio statistics.
    """
    rets = []   # List to store portfolio returns
    vols = []   # List to store portfolio volatilities
    wts = []    # List to store portfolio weights

    # Simulate portfolios
    for _ in range(numofportfolio):
        weights = np.random.random(numofasset)[:, np.newaxis]
        weights /= np.sum(weights)
        
        rets.append(weights.T @ np.array(returns.mean() * 252)[:, np.newaxis])        
        vols.append(np.sqrt(np.linalg.multi_dot([weights.T, returns.cov() * 252, weights])))
        wts.append(weights.flatten())

    # Create a DataFrame for analysis
    portdf = 100 * pd.DataFrame({
        'port_rets': np.array(rets).flatten(),
        'port_vols': np.array(vols).flatten(),
        'weights': list(np.array(np.round(wts, 4)))
    })
    
    portdf['sharpe_ratio'] = portdf['port_rets'] / portdf['port_vols']

    return round(portdf, 2)

In [ ]:
# Perform portfolio simulation
temp = portfolio_simulation(returns)

In [ ]:
# Get the portfolio with the maximum Sharpe ratio
max_sharpe_portfolio = temp.iloc[temp.sharpe_ratio.idxmax()]

In [ ]:
# Display statistics for the maximum Sharpe ratio portfolio
print("Maximum Sharpe Ratio Portfolio:")
print("-------------------------------")
print(max_sharpe_portfolio)

In [ ]:
# Create the scatter plot
fig_scatter = go.Figure()

# Add markers for all simulated portfolios
fig_scatter.add_trace(go.Scatter(
    x=temp['port_vols'],
    y=temp['port_rets'],
    mode='markers',
    marker=dict(
        size=5,
        color=temp['sharpe_ratio'],
        colorscale='Viridis',
        showscale=True
    ),
    text=temp.index
))

# Add marker for the maximum Sharpe ratio portfolio
fig_scatter.add_trace(go.Scatter(
    x=[max_sharpe_portfolio['port_vols']],
    y=[max_sharpe_portfolio['port_rets']],
    mode='markers',
    marker=dict(size=10, color='red', symbol='star'),
    name='Max Sharpe Ratio Portfolio'
))

# Update layout and axes labels
fig_scatter.update_layout(
    title="Portfolio Optimization",
    xaxis_title="Expected Volatility",
    yaxis_title="Expected Return",
    coloraxis_colorbar_title="Sharpe Ratio",
    showlegend=False,
    height=700,
    width=900
)

# Show the scatter plot
fig_scatter.show()

---
### <center>END OF CODE<center>